<a href="https://colab.research.google.com/github/WadeShadow/nlp2021/blob/main/Lab3/Lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np, pandas as pd, tensorflow as tf, requests as rqst, io
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras import layers

rnd = np.random.randint

folder = '/content/drive/MyDrive/Colab Notebooks/NLP_labs/'

## На основі підготовлених у попередній лабораторній роботі речень кожного з учасників створити загальний набір даних розмірністю (100, n), де n -кількість учасників команди, та відповідні мітки класів до них


In [2]:
RL_file = open(folder+'sentences.txt')
RL_sentences = RL_file.read().split('. ')[:-1]

DARINA_file = open(folder+'100.txt', encoding='windows-1251')
DARINA_sente                                                                                                                                                                                                  nces = DARINA_file.read().split('. ')[:-1]

RL_sentences

['Training a virtual agent to outperform human players can teach us how to optimize different processes in a variety of different and exciting subfields',
 'This is what Google DeepMind did with its popular AlphaGo, which beat the strongest Go player in history and scored a goal that was considered impossible at the time.In this article, we will develop an AI agent that is able to learn how to play the popular game Snake from scratch',
 'To do it, we implement a Deep Reinforcement Learning algorithm using both Keras on top of Tensorflow and PyTorch (both versions are available, you can choose the one you prefer)',
 'This approach consists in the interaction between two components: an environment (the game itself) and an agent (Snake)',
 'The agent collects information about its current state (we will see later what this means) and performs an action accordingly',
 'The environment rewards or punishes the agent based on the performed action',
 'Over time, the agent learns what actions m

In [3]:
sent_df = []

for sent in RL_sentences:
  sent_df.append({'Sentence':sent, 'Target':'Reinforcement Learning'})

for sent in DARINA_sentences:
  sent_df.append({'Sentence':sent, 'Target':'Neural network module for recognizing brain diseases'})

sent_df = pd.DataFrame(data=sent_df, columns=['Sentence', 'Target'])

sent_df['Label'] = sent_df['Target']
sent_df.replace({'Label':{'Reinforcement Learning':0, 'Neural network module for recognizing brain diseases':1}}, inplace=True)
sent_df

,Sentence,Target,Label
0,Training a virtual agent to outperform human p...,Reinforcement Learning,0
1,This is what Google DeepMind did with its popu...,Reinforcement Learning,0
2,"To do it, we implement a Deep Reinforcement Le...",Reinforcement Learning,0
3,This approach consists in the interaction betw...,Reinforcement Learning,0
4,The agent collects information about its curre...,Reinforcement Learning,0
...,...,...,...
211,This includes balancing the prospective benefi...,Neural network module for recognizing brain di...,1
212,"Users report their symptoms into the app, whic...",Neural network module for recognizing brain di...,1
213,Most common users could be people with serious...,Neural network module for recognizing brain di...,1
214,Body scans can spot cancer and vascular diseas...,Neural network module for recognizing brain di...,1


## Здійснити підготовку набору даних до моделювання (tokenization&embedding, train_test_split)


In [4]:
max_tokens = 10000

sentences = RL_sentences+DARINA_sentences
tokens_count = 0
for sent in sentences:
  tokens_count+=len(sent.split())
avg_tokens = round(tokens_count/len(sentences))
avg_tokens

23

#### Tokenization&Embedding

In [5]:
text_vectorizer = TextVectorization(max_tokens=max_tokens, 
                                    standardize="lower_and_strip_punctuation", 
                                    split="whitespace", 
                                    ngrams=None, 
                                    output_mode="int",
                                    output_sequence_length=avg_tokens, 
                                    pad_to_max_tokens=True)

text_vectorizer.adapt(sent_df['Sentence'])

text_vectorizer(sent_df['Sentence'])

<tf.Tensor: shape=(216, 23), dtype=int64, numpy=
array([[ 108,    6,  340, ...,  120,    4, 1214],
       [  27,    8,   47, ...,    6,   76,    9],
       [   3,  152,   26, ...,  618,   22,  216],
       ...,
       [  86,  314,  342, ...,    7,   64,  126],
       [1387,  794,   20, ..., 1159,    0,    0],
       [ 244,    2,  119, ...,  446,    8,    2]])>

In [6]:
embedding = layers.Embedding(input_dim=max_tokens, # set input shape
                             output_dim=128, # set size of embedding vector
                             embeddings_initializer="uniform", # default, intialize randomly
                             input_length=avg_tokens) # how long is each input

embedding(text_vectorizer(sent_df['Sentence']))

<tf.Tensor: shape=(216, 23, 128), dtype=float32, numpy=
array([[[ 6.97020441e-03, -1.96845662e-02,  2.09697820e-02, ...,
          2.98492350e-02,  3.70584168e-02,  4.55253161e-02],
        [ 2.17731483e-02,  2.40580551e-02,  2.60901712e-02, ...,
          3.12251560e-02,  4.39117886e-02, -4.66168411e-02],
        [ 3.49115767e-02,  2.40859427e-02, -1.88845154e-02, ...,
          6.48991019e-03,  2.00299658e-02, -1.83723196e-02],
        ...,
        [ 1.94359459e-02,  1.20611675e-02,  3.19797881e-02, ...,
         -3.55614051e-02, -1.26329772e-02,  3.27330939e-02],
        [-1.85273066e-02, -1.97102558e-02,  1.75294317e-02, ...,
          3.26038115e-02,  3.20647024e-02,  4.60136421e-02],
        [ 1.82113312e-02,  5.48766926e-03,  5.05348295e-03, ...,
          4.72555496e-02,  3.57601792e-03, -4.55062389e-02]],

       [[ 1.67846680e-03, -4.18853052e-02,  4.99509089e-02, ...,
          1.70918144e-02, -1.95655227e-02, -1.42388716e-02],
        [-4.07283306e-02, -1.44096985e-02,  3.3

#### Train/Test Split

In [7]:
from sklearn.model_selection import train_test_split
train_split, test_split = train_test_split(sent_df, train_size=0.8, test_size=0.2)
train_split

,Sentence,Target,Label
82,Similar algorithms in principal can be used to...,Reinforcement Learning,0
104,"Based on the feedback it receives, the agent l...",Reinforcement Learning,0
206,"Also, by ensuring a continuous feedback loop m...",Neural network module for recognizing brain di...,1
132,"Furthermore, the amount of medical data is con...",Neural network module for recognizing brain di...,1
102,Based on how much those actions affect the goa...,Reinforcement Learning,0
...,...,...,...
145,The unraveling of underlying affected genetic ...,Neural network module for recognizing brain di...,1
12,"Robotics, Autonomous driving, etc..) and Decis...",Reinforcement Learning,0
200,The research and results of these tests are st...,Neural network module for recognizing brain di...,1
131,"This highly complex process has, on the one ha...",Neural network module for recognizing brain di...,1


## Вирішити завдання мультикласової класифікації на створеному наборі даних із використанням методів: Naive Bayes Model, Dense Neural Network

In [12]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
  -----
  y_true = true labels in the form of a 1D array
  y_pred = predicted labels in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall, f1-score.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results


### Naive Bayes Model

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf
                    ("clf", MultinomialNB()) # model the text
])

# Fit the pipeline to the training data
model_0.fit(train_split['Sentence'], train_split['Label'])
baseline_score = model_0.score(test_split['Sentence'], test_split['Label'])
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 86.36%


### Dense Neural Network

In [9]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string") # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numerized numbers
x = layers.GlobalAveragePooling1D()(x) # lower the dimensionality of the embedding (try running the model without this layer and see what happens)
outputs = layers.Dense(1, activation="sigmoid")(x) # create the output layer, want binary outputs so use sigmoid activation
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense") # construct the model


In [10]:
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

model_1.summary()


Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 23)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 23, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [11]:
# Fit the model
model_1_history = model_1.fit(train_split['Sentence'], # input sentences can be a list of strings due to text preprocessing layer built-in model
                              train_split['Label'],
                              epochs=50,
                              validation_data=(test_split['Sentence'], test_split['Label']))

Epoch 1/50
6/6 [==============================] - 3s 56ms/step - loss: 0.6899 - accuracy: 0.6047 - val_loss: 0.6844 - val_accuracy: 0.6364
Epoch 2/50
6/6 [==============================] - 0s 13ms/step - loss: 0.6760 - accuracy: 0.8023 - val_loss: 0.6766 - val_accuracy: 0.6818
Epoch 3/50
6/6 [==============================] - 0s 12ms/step - loss: 0.6634 - accuracy: 0.8256 - val_loss: 0.6686 - val_accuracy: 0.6591
Epoch 4/50
6/6 [==============================] - 0s 12ms/step - loss: 0.6500 - accuracy: 0.8488 - val_loss: 0.6600 - val_accuracy: 0.6818
Epoch 5/50
6/6 [==============================] - 0s 12ms/step - loss: 0.6348 - accuracy: 0.8547 - val_loss: 0.6514 - val_accuracy: 0.6818
Epoch 6/50
6/6 [==============================] - 0s 11ms/step - loss: 0.6194 - accuracy: 0.8721 - val_loss: 0.6426 - val_accuracy: 0.7045
Epoch 7/50
6/6 [==============================] - 0s 12ms/step - loss: 0.6026 - accuracy: 0.8837 - val_loss: 0.6326 - val_accuracy: 0.7500
Epoch 8/50
6/6 [===========

## Здійснити порівняння ефективності моделей

In [18]:
calculate_results(test_split['Label'], model_0.predict(test_split['Sentence']))

{'accuracy': 86.36363636363636,
 'f1': 0.8601398601398601,
 'precision': 0.8918495297805642,
 'recall': 0.8636363636363636}

In [19]:
calculate_results(test_split['Label'], tf.squeeze(tf.round(model_1.predict(test_split['Sentence']))))


{'accuracy': 88.63636363636364,
 'f1': 0.8854716711859569,
 'precision': 0.8930652680652681,
 'recall': 0.8863636363636364}